In [1]:
import pandas as pd
from os.path import expanduser, join
from glob import glob

In [2]:
user = expanduser('~')
fpath = join(user, 'Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1')
dest = join(fpath, 'Analysis')
datadir = join(fpath, 'DataDir_qT1/5_Extracted_Stats')
flist = glob(datadir + '/*/ROI_voxel-*')

In [7]:
def fix_files(file1):
    inf = file1.replace('/Users/lucindasisk/Dropbox/SNAPLab/Data/TIGER-T1/TIGER_qT1/DataDir_qT1/5_Extracted_Stats/_subject_id_', '').replace('/ROI_voxel-contrasts', '').replace('.txt', '')
    file1 = pd.read_csv(file1, header=None, sep = '  ')
    file = file1.transpose().rename(columns = {0:'L/R', 1:'A/P', 2:'S/I', 3:'Voxel Contrast'})
    file['R1'] = 1/file['Voxel Contrast']
    outfile = file.to_csv(dest + '/{}.csv'.format(inf), index = False)
    return outfile

for file1 in flist:
    fix_files(file1)


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
